
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%run ../Includes/Copy-Datasets

In [0]:
%fs ls 'dbfs:/mnt/demo-datasets/bookstore/orders'

path,name,size,modificationTime
dbfs:/mnt/demo-datasets/bookstore/orders/export_001.parquet,export_001.parquet,22279,1746436841000
dbfs:/mnt/demo-datasets/bookstore/orders/export_002.parquet,export_002.parquet,16659,1746436842000
dbfs:/mnt/demo-datasets/bookstore/orders/export_003.parquet,export_003.parquet,11842,1746436843000


In [0]:
%sql
CREATE OR REPLACE TABLE orders AS
SELECT * FROM parquet.`${dataset.bookstore}/orders`

num_affected_rows,num_inserted_rows


In [0]:
%sql
describe extended orders

col_name,data_type,comment
order_id,string,null
order_timestamp,bigint,null
customer_id,string,null
quantity,bigint,null
total,int,null
books,array>,null
,,
# Detailed Table Information,,
Catalog,hive_metastore,
Database,default,


In [0]:
%sql
SELECT * FROM orders LIMIT 4

order_id,order_timestamp,customer_id,quantity,total,books
000000000003559,1657722056,C00001,2,48,"List(List(B09, 2, 48))"
000000000004243,1658786901,C00002,2,55,"List(List(B07, 1, 33), List(B06, 1, 22))"
000000000004321,1658934252,C00003,2,40,"List(List(B04, 2, 40))"
000000000004392,1659034513,C00004,2,82,"List(List(B08, 2, 82))"


## Overwriting Tables

In [0]:
%sql
CREATE OR REPLACE TABLE orders AS
SELECT * FROM parquet.`${dataset.bookstore}/orders`

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE HISTORY orders

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-05-05T10:24:19.000+0000,983714032662280,luubaonluu@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(763081737064110),0505-091253-qv84mjl5,0,WriteSerializable,false,"Map(numFiles -> 3, numOutputRows -> 2150, numOutputBytes -> 50780)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-05-05T10:22:37.000+0000,983714032662280,luubaonluu@gmail.com,CREATE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(763081737064110),0505-091253-qv84mjl5,null,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 2150, numOutputBytes -> 50780)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql
INSERT OVERWRITE orders
SELECT * FROM parquet.`${dataset.bookstore}/orders`

num_affected_rows,num_inserted_rows
2150,2150


In [0]:
%sql
DESCRIBE HISTORY orders

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-05-05T10:25:11.000+0000,983714032662280,luubaonluu@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(763081737064110),0505-091253-qv84mjl5,1,WriteSerializable,false,"Map(numFiles -> 3, numOutputRows -> 2150, numOutputBytes -> 50780)",null,Databricks-Runtime/12.2.x-scala2.12
1,2025-05-05T10:24:19.000+0000,983714032662280,luubaonluu@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(763081737064110),0505-091253-qv84mjl5,0,WriteSerializable,false,"Map(numFiles -> 3, numOutputRows -> 2150, numOutputBytes -> 50780)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-05-05T10:22:37.000+0000,983714032662280,luubaonluu@gmail.com,CREATE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(763081737064110),0505-091253-qv84mjl5,null,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 2150, numOutputBytes -> 50780)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql
INSERT OVERWRITE orders
SELECT *, current_timestamp() FROM parquet.`${dataset.bookstore}/orders`

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-763081737064120>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-763081737064120>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("SU5TRVJUIE9WRVJXUklURSBvcmRlcnMKU0VMRUNUICosIGN1cnJlbnRfdGltZXN0YW1wKCkgRlJPTSBwYXJxdWV0LmAke2RhdGFzZXQuYm9va3N0b3JlfS9vcmRlcnNg").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name

## Appending Data

In [0]:
%sql
INSERT INTO orders
SELECT * FROM parquet.`${dataset.bookstore}/orders-new`

In [0]:
%sql
SELECT count(*) FROM orders

## Merging Data

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW customers_updates AS 
SELECT * FROM json.`${dataset.bookstore}/customers-json-new`;

MERGE INTO customers c
USING customers_updates u
ON c.customer_id = u.customer_id
WHEN MATCHED AND c.email IS NULL AND u.email IS NOT NULL THEN
  UPDATE SET email = u.email, updated = u.updated
WHEN NOT MATCHED THEN INSERT *

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW books_updates
   (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS (
  path = "${dataset.bookstore}/books-csv-new",
  header = "true",
  delimiter = ";"
);

SELECT * FROM books_updates

In [0]:
%sql
MERGE INTO books b
USING books_updates u
ON b.book_id = u.book_id AND b.title = u.title
WHEN NOT MATCHED AND u.category = 'Computer Science' THEN 
  INSERT *